# 1. Preprocess data set and performing EDA

In this section we have described how we will preprocess data set by reading files from S3
- Input: S3 files in csv format 
- Output: Preprocess data which can be used for model training 

In [1]:
# Importing required library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
##option--Installing Kaggle API for data set download##
! pip install kaggle --upgrade
#kaggle competitions download -c titanic

In [ ]:
!kaggle competitions download -c titanic

In [ ]:
# import boto3, os
# boto3.Session().resource('s3').Bucket('poc-sagemaker-step-function-bucketformodelanddata-iip9wy7qh0y0').Object(os.path.join('train', 'train_data.csv')).upload_file('train_data.csv')

In [87]:
df = pd.read_csv("train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [89]:
!aws s3 cp train.csv s3://ds-mlops-dev/titanic/data/input/inputdata.csv

upload: ./train.csv to s3://ds-mlops-dev/titanic/data/input/inputdata.csv


In [86]:
!pwd

/home/ec2-user/SageMaker/WipCoe/Notebooks_LatestCode/WiproFormat


describing dataset which we have got

In [ ]:
df.describe()

Removing unnessary columns

In [ ]:
# look at numeric and categorical values separately 
df_num = df[['Age','SibSp','Parch','Fare']]
df_cat = df[['Survived','Pclass','Sex','Ticket','Cabin','Embarked']]

Imputing data for missing values

In [ ]:
#distributions for all numeric variables 
from matplotlib import pyplot as plt
for i in df_num.columns:
    plt.hist(df_num[i])
    plt.title(i)
    plt.show()

In [ ]:
import seaborn as sns
print(df_num.corr())
sns.heatmap(df_num.corr(),annot=True)

In [ ]:
# compare survival rate across Age, SibSp, Parch, and Fare 
pd.pivot_table(df, index = 'Survived', values = ['Age','SibSp','Parch','Fare'])

In [ ]:
for i in df_cat.columns:
    sns.barplot(df_cat[i].value_counts().index,df_cat[i].value_counts()).set_title(i)
    plt.show()

In [ ]:
# Cabin - Simplify cabins (evaluated if cabin letter (cabin_adv) or the purchase of tickets across multiple cabins (cabin_multiple) impacted survival)

# Tickets - Do different ticket types impact survival rates?

# Does a person's title relate to survival rates?

In [ ]:
df_cat.Cabin
df['cabin_multiple'] = df.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))
# after looking at this, we may want to look at cabin by letter or by number. Let's create some categories for this 
# letters 
# multiple letters 
df['cabin_multiple'].value_counts()

In [ ]:
# Data Preprocessing for Model
# Drop null values from Embarked (only 2)

# Include only relevant variables (Since we have limited data, I wanted to exclude things like name and passanger ID so that we could have a reasonable number of features for our models to deal with) Variables: 'Pclass', 'Sex','Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'cabin_adv', 'cabin_multiple', 'numeric_ticket', 'name_title'

# Do categorical transforms on all data. Usually we would use a transformer, but with this approach we can ensure that our traning and test data have the same colums. We also may be able to infer something about the shape of the test data through this method. I will stress, this is generally not recommend outside of a competition (use onehot encoder).

# Impute data with mean for fare and age (Should also experiment with median)

# Normalized fare using logarithm to give more semblance of a normal distribution

# Scaled data 0-1 with standard scaler

In [ ]:
df['train_test'] = 1
#df_test=pd.read_csv("test.csv")
#df_test['train_test'] = 0
#df_test['Survived'] = np.NaN
all_data = pd.concat([df])

In [ ]:
#create all categorical variables that we did above for both training and test sets 
all_data['cabin_multiple'] = all_data.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))
all_data['cabin_adv'] = all_data.Cabin.apply(lambda x: str(x)[0])
all_data['numeric_ticket'] = all_data.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
all_data['ticket_letters'] = all_data.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1]).replace('.','').replace('/','').lower() if len(x.split(' ')[:-1]) >0 else 0)
all_data['name_title'] = all_data.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())
#impute nulls for continuous data 
#all_data.Age = all_data.Age.fillna(training.Age.mean())
all_data.Age = all_data.Age.fillna(df.Age.median())
#all_data.Fare = all_data.Fare.fillna(training.Fare.mean())
all_data.Fare = all_data.Fare.fillna(df.Fare.median())
#drop null 'embarked' rows. Only 2 instances of this in training and 0 in test 
all_data.dropna(subset=['Embarked'],inplace = True)
#tried log norm of sibsp (not used)
all_data['norm_sibsp'] = np.log(all_data.SibSp+1)
all_data['norm_sibsp'].hist()
# log norm of fare (used)
all_data['norm_fare'] = np.log(all_data.Fare+1)
all_data['norm_fare'].hist()
# converted fare to category for pd.get_dummies()
all_data.Pclass = all_data.Pclass.astype(str)
#created dummy variables from categories (also can use OneHotEncoder)
all_dummies = pd.get_dummies(all_data[['Survived','Pclass','Sex','Age','SibSp','Parch','norm_fare','Embarked','cabin_adv','cabin_multiple','numeric_ticket','name_title']])
#Split to train test again


In [ ]:
all_dummies

In [34]:
# Scale data 
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
encoded_df = all_dummies.copy()
encoded_df[['Age','SibSp','Parch','norm_fare']]= scale.fit_transform(all_dummies_scaled[['Age','SibSp','Parch','norm_fare']])
encoded_df

,Survived,Age,SibSp,Parch,norm_fare,cabin_multiple,numeric_ticket,Pclass_1,Pclass_2,Pclass_3,...,name_title_Master,name_title_Miss,name_title_Mlle,name_title_Mme,name_title_Mr,name_title_Mrs,name_title_Ms,name_title_Rev,name_title_Sir,name_title_the Countess
0,0,-0.563674,0.431350,-0.474326,-0.877591,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
1,1,0.669217,0.431350,-0.474326,1.366380,1,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,-0.255451,-0.475199,-0.474326,-0.796281,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
3,1,0.438050,0.431350,-0.474326,1.066796,1,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0.438050,-0.475199,-0.474326,-0.781901,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,-0.178396,-0.475199,-0.474326,-0.330815,0,1,0,1,0,...,0,0,0,0,0,0,0,1,0,0
887,1,-0.794841,-0.475199,-0.474326,0.491068,1,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
888,0,-0.101340,0.431350,2.006119,0.245663,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
889,1,-0.255451,-0.475199,-0.474326,0.491068,1,1,1,0,0,...,0,0,0,0,1,0,0,0,0,0


We'll then split the dataset into training (70%), validation (20%), and test (10%) datasets and convert the datasets to the right format the algorithm expects. We will use training and validation datasets during training. Test dataset will be used to evaluate model performance after it is deployed to an endpoint.

Amazon SageMaker's XGBoost algorithm expects data in the libSVM or CSV data format.  For this example, we'll stick to CSV.  Note that the first column must be the target variable and the CSV should not include headers.  Also, notice that although repetitive it's easiest to do this after the train|validation|test split rather than before.  This avoids any misalignment issues due to random reordering.

In [35]:
train_data, validation_data, test_data = np.split(encoded_df.sample(frac=1, random_state=1729), [int(0.7 * len(encoded_df)), int(0.9*len(encoded_df))])  

In [36]:
train_data.shape

(622, 41)

In [37]:
validation_data.shape

(178, 41)

In [38]:
test_data.shape

(89, 41)

**Exporting data set to s3 bucket without header**
## please change below S3 path as per the VW bucket name and folder location

In [39]:
train_data.to_csv('s3://wi-cred-datalake-dev-raw/titanic/input/lrtrain/train.csv', index=False, header=False)
validation_data.to_csv('s3://wi-cred-datalake-dev-raw/titanic/input/lrvalidation/validation_data.csv', index=False, header=False)
test_data.to_csv('s3://wi-cred-datalake-dev-raw/titanic/input/lrtest/test.csv', index=False, header=False)

## 2. Building model

In this section we will decribe how we will build model with XGboost inbuilt model and here we will pass hyperparameter to train our model

In [51]:
# Importing the required libraries
import boto3
import time
from time import gmtime, strftime
import sagemaker
from sagemaker.amazon.amazon_estimator import get_image_uri
import datetime
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.inputs import TrainingInput
# sagemaker
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
#ec2
Instance_Type = "ml.m5.xlarge"
VolumeSizeIn_GB = 1
Instance_Count = 1
#setup container
container = get_image_uri(region, "linear-learner")
#s3
#################
#- Please change as per the relevant S3 bucket name
bucket = "wi-cred-datalake-dev-raw"
prefix = "titanic/input"
##############
#input
data_bucket = bucket
data_prefix = prefix
data_bucket_path = f"s3://{data_bucket}"
# S3 bucket for saving code and model artifacts.
# output
output_bucket = sagemaker.Session().default_bucket()
output_prefix = 'titanic/output'
output_bucket_path = f"s3://{output_bucket}"

##Plase change as per VW Config
#sec_groups = ["sg-044e0e7ce4f5721c0"]
#subnets = ["subnet-0cf0e3f46326aa259","subnet-0156b7f5500cf0b78","subnet-032420199163cff9b"]

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


In [41]:
print(role)

arn:aws:iam::525102048888:role/SagemakerCustomRoleForForecast


---

## Setup_Hyperparameter_Tuning 
*Note, with the default setting below, the hyperparameter tuning job can take about 30 minutes to complete.*


Now that we have prepared the dataset, we are ready to train models. Before we do that, one thing to note is there are algorithm settings which are called "hyperparameters" that can dramtically affect the performance of the trained models. For example, XGBoost algorithm has dozens of hyperparameters and we need to pick the right values for those hyperparameters in order to achieve the desired model training results. Since which hyperparameter setting can lead to the best result depends on the dataset as well, it is almost impossible to pick the best hyperparameter setting without searching for it, and a good search algorithm can search for the best hyperparameter setting in an automated and effective way.

We will use SageMaker hyperparameter tuning to automate the searching process effectively. Specifically, we specify a range, or a list of possible values in the case of categorical hyperparameters, for each of the hyperparameter that we plan to tune. SageMaker hyperparameter tuning will automatically launch multiple training jobs with different hyperparameter settings, evaluate results of those training jobs based on a predefined "objective metric", and select the hyperparameter settings for future attempts based on previous results. For each hyperparameter tuning job, we will give it a budget (max number of training jobs) and it will complete once that many training jobs have been executed.

In this example, we are using SageMaker Python SDK to set up and manage the hyperparameter tuning job. We first configure the training jobs the hyperparameter tuning job will launch by initiating an estimator, which includes:
* The container image for the algorithm (XGBoost)
* Configuration for the output of the training jobs
* The values of static algorithm hyperparameters, those that are not specified will be given default values
* The type and number of instances to use for the training jobs

In [44]:
#setup sagemaker SDK
client = boto3.client("sagemaker", region_name=region)

In [45]:
sess = sagemaker.Session()
linear = sagemaker.estimator.Estimator(container,
                                       role, 
                                       instance_count = 1, 
                                       instance_type = 'ml.c4.xlarge',
                                       output_path = output_bucket_path,
                                       sagemaker_session = sess)

linear.set_hyperparameters(
                           predictor_type = 'regressor',
                           mini_batch_size = 20,
                           epochs = 50,
                           num_models = 10,
                           loss = 'absolute_loss')

We will tune four hyperparameters in this examples:
* *eta*: Step size shrinkage used in updates to prevent overfitting. After each boosting step, you can directly get the weights of new features. The eta parameter actually shrinks the feature weights to make the boosting process more conservative. 
* *alpha*: L1 regularization term on weights. Increasing this value makes models more conservative. 
* *min_child_weight*: Minimum sum of instance weight (hessian) needed in a child. If the tree partition step results in a leaf node with the sum of instance weight less than min_child_weight, the building process gives up further partitioning. In linear regression models, this simply corresponds to a minimum number of instances needed in each node. The larger the algorithm, the more conservative it is. 
* *max_depth*: Maximum depth of a tree. Increasing this value makes the model more complex and likely to be overfitted. 

In [46]:
hyperparameter_ranges = {'mini_batch_size': CategoricalParameter([32,64,128,256,512]),
                        'learning_rate': ContinuousParameter(0.0001, 1),
                        'l1': ContinuousParameter(0.0001, 1)
                        }

Next we'll specify the objective metric that we'd like to tune and its definition, which includes the regular expression (Regex) needed to extract that metric from the CloudWatch logs of the training job. Since we are using built-in XGBoost algorithm here, it emits two predefined metrics: *validation:auc* and *train:auc*, and we elected to monitor *validation:auc* as you can see below. In this case, we only need to specify the metric name and do not need to provide regex. If you bring your own algorithm, your algorithm emits metrics by itself. In that case, you'll need to add a MetricDefinition object here to define the format of those metrics through regex, so that SageMaker knows how to extract those metrics from your CloudWatch logs.

In [47]:
objective_metric_name = 'validation:objective_loss:final'
objective_type = 'Minimize'

Now, we'll create a `HyperparameterTuner` object, to which we pass:
- The XGBoost estimator we created above
- Our hyperparameter ranges
- Objective metric name and definition
- Tuning resource configurations such as Number of training jobs to run in total and how many training jobs can be run in parallel.

In [48]:
tuner = HyperparameterTuner(linear,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=1,
                            max_parallel_jobs=1,
                            early_stopping_type='Auto',
                            objective_type=objective_type)

## Launch_Hyperparameter_Tuning
Now we can launch a hyperparameter tuning job by calling *fit()* function. After the hyperparameter tuning job is created, we can go to SageMaker console to track the progress of the hyperparameter tuning job until it is completed.

In [52]:
print('{}/{}/lrtrain/train.csv'.format(data_bucket_path, data_prefix))
print('{}/{}/lrvalidation/validation_data.csv'.format(data_bucket_path, data_prefix))

s3://wi-cred-datalake-dev-raw/titanic/input/lrtrain/train.csv
s3://wi-cred-datalake-dev-raw/titanic/input/lrvalidation/validation_data.csv


In [53]:
s3_input_train = TrainingInput(s3_data='{}/{}/lrtrain/train.csv'.format(data_bucket_path, data_prefix), content_type='text/csv')
s3_input_validation = TrainingInput(s3_data='{}/{}/lrvalidation/validation_data.csv'.format(data_bucket_path, data_prefix), content_type='text/csv')
tuner.fit({'train': s3_input_train, 'validation': s3_input_validation}, include_cls_metadata=False)

..................................................!


Let's just run a quick check of the hyperparameter tuning jobs status to make sure it started successfully.

In [54]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

'Completed'

In [ ]:
print(data_bucket_path,data_prefix)

In [55]:
tuning_job_name = tuner.latest_tuning_job.job_name # Get name of hpo job

# Analyze Results of a Hyperparameter Tuning job

Once you have completed a tuning job, (or even while the job is still running) you can use this notebook to analyze the results to understand how each hyperparameter effects the quality of the model.


## Track hyperparameter tuning job progress and print best result
After you launch a tuning job, you can see its progress by calling describe_tuning_job API. The output from describe-tuning-job is a JSON object that contains information about the current state of the tuning job. You can call list_training_jobs_for_tuning_job to see a detailed list of the training jobs that the tuning job launched.

In [56]:
# run this cell to check current status of hyperparameter tuning job
region = boto3.Session().region_name
sage_client = boto3.Session().client('sagemaker')
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)


status = tuning_job_result['HyperParameterTuningJobStatus']
if status != 'Completed':
    print('Reminder: the tuning job has not been completed.')
    
job_count = tuning_job_result['TrainingJobStatusCounters']['Completed']
print("%d training jobs have completed" % job_count)

best_hyper_parameter = tuning_job_result['BestTrainingJob']['TunedHyperParameters']
print("Best performing hyper parameters :",best_hyper_parameter)

result = tuning_job_result['BestTrainingJob']['FinalHyperParameterTuningJobObjectiveMetric']
print("Result : ",result)

1 training jobs have completed
Best performing hyper parameters : {'l1': '0.0014121036264995773', 'learning_rate': '0.012104062065442999', 'mini_batch_size': '256'}
Result :  {'MetricName': 'validation:objective_loss:final', 'Value': 0.2061675488948822}


In [33]:
best_hyper_parameter={'l1': '0.0014121036264995773', 'learning_rate': '0.012104062065442999', 'mini_batch_size': '256'}
ll_best_hyper_parameter = best_hyper_parameter


In [58]:
print(ll_best_hyper_parameter['l1'],ll_best_hyper_parameter['learning_rate'],ll_best_hyper_parameter['mini_batch_size'])
print(ll_best_hyper_parameter)

0.0014121036264995773 0.012104062065442999 256
{'l1': '0.0014121036264995773', 'learning_rate': '0.012104062065442999', 'mini_batch_size': '256'}


In [34]:
%store ll_best_hyper_parameter

Stored 'll_best_hyper_parameter' (dict)


**Creating model for Linear Learner by passing hyper parameter**

Once we have tunned parameter we will generate our model for scoring activities on sagemaker

In [ ]:
{output_bucket_path},{output_prefix}

In [62]:
%%time
job_name = 'titanic-linear-learner' + str(datetime.datetime.now().timestamp()).split('.')[0]
print("Training job", job_name)

# Ensure that the training and validation data folders generated above are reflected in the "InputDataConfig" parameter below.
create_training_params = {
    "AlgorithmSpecification": {"TrainingImage": container, "TrainingInputMode": "File"},
    "RoleArn": role,
    "OutputDataConfig": {"S3OutputPath": f"{output_bucket_path}/{output_prefix}/output"},
    "ResourceConfig": {"InstanceCount": Instance_Count, "InstanceType": Instance_Type, "VolumeSizeInGB": VolumeSizeIn_GB},
    "TrainingJobName": job_name,
    "HyperParameters": {"predictor_type" :'regressor',
                        "epochs":"20",
                        "l1":ll_best_hyper_parameter['l1'],
                        "learning_rate":ll_best_hyper_parameter['learning_rate'],
                        "mini_batch_size":ll_best_hyper_parameter['mini_batch_size']
                       },
    "StoppingCondition": {"MaxRuntimeInSeconds": 3600},
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": f"s3://wi-cred-datalake-dev-raw/titanic/input/lrtrain/train.csv",
                    "S3DataDistributionType": "FullyReplicated",
                }
            },
            "ContentType": "text/csv",
            "CompressionType": "None",
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri":f"s3://wi-cred-datalake-dev-raw/titanic/input/lrvalidation/validation_data.csv",
                    "S3DataDistributionType": "FullyReplicated",
                }
            },
            "ContentType": "text/csv",
            "CompressionType": "None",
            
        },
    ],
}

client.create_training_job(**create_training_params)

#track execution
status = client.describe_training_job(TrainingJobName=job_name)["TrainingJobStatus"]
print(status)
while status != "Completed" and status != "Failed":
    time.sleep(60)
    status = client.describe_training_job(TrainingJobName=job_name)["TrainingJobStatus"]
    print(status)

Training job titanic-linear-learner1659327776
InProgress
InProgress
InProgress
Completed
CPU times: user 66.8 ms, sys: 1.18 ms, total: 68 ms
Wall time: 3min


In [63]:
# checking training job status after creating training job
status = client.describe_training_job(TrainingJobName=job_name)["TrainingJobStatus"]
print(status)

Completed


In [64]:
# printing model output location
print(f"model path: >> {output_bucket_path}/{output_prefix}/output")

model path: >> s3://sagemaker-us-east-1-525102048888/titanic/output/output


In [ ]:
# printing the training metrics you can tune and generate model as per need
metrics = client.describe_training_job(TrainingJobName=job_name)["FinalMetricDataList"]
print(metrics)

In [ ]:
# finding job name 
print(job_name)

**Note:** you will need folowing values in next section
- hyperparameter
- training job name

## 3. Generate model on Sagemaker

In section we will describe how we can create a model with our training jobs

In [65]:
import sys
import math
import boto3
import sagemaker
from sagemaker.amazon.amazon_estimator import get_image_uri
from time import gmtime, strftime

model_name = 'Titanic-linear-learner' # It need to be unique across the account

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
runtime_client = boto3.client("runtime.sagemaker", 
                              region_name=region)
client = boto3.client("sagemaker", 
                      region_name=region)

Check if model with same name exists or not. If it exist then delete it.

In [66]:
client = boto3.client('sagemaker') # getting sagemaker client 
try:
    client.delete_model(
        ModelName=model_name # delete if some model exist with this name
    )
except:
    pass

In [67]:
container = get_image_uri(region, "linear-learner") # getting container image

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


**Register model**


In [68]:
# Here we are registering our model
print(model_name) 
info = client.describe_training_job(TrainingJobName=job_name)
model_data = info["ModelArtifacts"]["S3ModelArtifacts"]
print(model_data)
primary_container = {"Image": container, "ModelDataUrl": model_data}
create_model_response = client.create_model(
    ModelName=model_name, 
    ExecutionRoleArn=role, 
    PrimaryContainer=primary_container
#     VpcConfig={
#             'SecurityGroupIds':sec_groups,
#             'Subnets': subnets
#             }
)
print(create_model_response["ModelArn"]) # printing model ARN which is cretaed

Titanic-linear-learner
s3://sagemaker-us-east-1-525102048888/titanic/output/output/titanic-linear-learner1659327776/output/model.tar.gz
arn:aws:sagemaker:us-east-1:525102048888:model/titanic-linear-learner


In [69]:
print(model_name)

Titanic-linear-learner


**Note :** 
- We will need this model ARN for generating endpoint 
- Also model name need to be unique in an aws account
- In case if we need to regenerate model we have to delete existing model

## 4. Delete endpoint config

In [70]:
import sys
import math
import boto3
import sagemaker
from time import gmtime, strftime

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
runtime_client = boto3.client("runtime.sagemaker", region_name=region)
client = boto3.client("sagemaker", region_name=region)
endpoint_config_name = 'Titanic-linear-learner-endpoint-config'

In [ ]:
client = boto3.client('sagemaker') # getting sagemaker client 
try:
    client.delete_endpoint_config(
     EndpointConfigName=endpoint_config_name
    )
except:
    pass


## 5. Create endpoint config
In this section we will describe how we can create a model endpoint.

For generating sagemaker end point we will need to generate endpoint config. So generating endpoint config first

In [71]:
import sys
import math
import boto3
import sagemaker
from time import gmtime, strftime

endpoint_config_name = 'Titanic-learner-endpoint-config'

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
runtime_client = boto3.client("runtime.sagemaker", region_name=region)
client = boto3.client("sagemaker", region_name=region)

In [72]:
# Instance config for endpoint
Instance_Type = "ml.m5.xlarge"
VolumeSizeIn_GB = 20
Instance_Count = 1

In [73]:
# Creating endpoint config
print(endpoint_config_name)
create_endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": Instance_Type,
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
    DataCaptureConfig={
        'EnableCapture': True,
        'InitialSamplingPercentage': 100,
        'DestinationS3Uri': 's3://wi-cred-datalake-dev-raw/capture/', #pls change as per VW settings
        'CaptureOptions': [
            {
                'CaptureMode': 'Input'
            },
             {
                'CaptureMode': 'Output'
            },
        ],
        'CaptureContentTypeHeader': {
            'CsvContentTypes': [
                'text/csv',
            ],
            'JsonContentTypes': [
                'application/json',
            ]
        }        
    }
)

print(f"Endpoint Config Arn: {create_endpoint_config_response['EndpointConfigArn']}")

Titanic-learner-endpoint-config
Endpoint Config Arn: arn:aws:sagemaker:us-east-1:525102048888:endpoint-config/titanic-learner-endpoint-config


**Note :** We will use able endpoint config name to cretae model endpoint

Incase if you want to delete endpoint config and recreate it we can use below code snippet

## 6.Create Endpoint

In this notebook we create endpoint using endpoint which we have created

In [74]:
#delete existing end point
endpoint_name = 'Titanic-linear-learner-endpoint'
client = boto3.client('sagemaker') # getting sagemaker client 
try:
    client.delete_endpoint(
     EndpointName=endpoint_name
    )
except:
    pass


In [75]:
# Importing required library
import sys
import math
import boto3
import sagemaker
import time    

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
runtime_client = boto3.client("runtime.sagemaker", region_name=region)
client = boto3.client("sagemaker", region_name=region)
endpoint_name = endpoint_name

In [76]:
# Creating model endpoint
print(endpoint_name)
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
while status == "Creating":
    print(f"Status: {status}")
    time.sleep(60)
    resp = client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]

print(f"Arn: {resp['EndpointArn']}")
print(f"Status: {status}")

Titanic-linear-learner-endpoint
arn:aws:sagemaker:us-east-1:525102048888:endpoint/titanic-linear-learner-endpoint
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Arn: arn:aws:sagemaker:us-east-1:525102048888:endpoint/titanic-linear-learner-endpoint
Status: InService


## 7. Validate scoring

In this section we will validate scoring logics. In below code snippet we are scoring with 14 predictor values and finding the result

In [79]:
validation_data.head()

,Survived,Age,SibSp,Parch,norm_fare,cabin_multiple,numeric_ticket,Pclass_1,Pclass_2,Pclass_3,...,name_title_Master,name_title_Miss,name_title_Mlle,name_title_Mme,name_title_Mr,name_title_Mrs,name_title_Ms,name_title_Rev,name_title_Sir,name_title_the Countess
506,1,0.283939,-0.475199,2.006119,0.348233,0,1,0,1,0,...,0,0,0,0,0,1,0,0,0,0
851,0,3.443222,-0.475199,-0.474326,-0.813806,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0
435,1,-1.180120,0.431350,2.006119,1.899046,2,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
747,1,0.052771,-0.475199,-0.474326,-0.330815,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0
681,1,-0.178396,-0.475199,-0.474326,1.441480,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [2]:
import boto3
import json
endpoint_name='wi-mlops-titanic-ml-train-piln-lr-endpoint'
client = boto3.client('sagemaker-runtime')
sur_test_vector = [-1.4112866455793727,0.43135023895238417,2.006119339191895,1.89904574757844,2,1,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0]
nonsur_test_vector=[0.6692169576775447,-0.47519908120995513,-0.474325852103786,-0.3308153928167703,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0]
body = ','.join([str(item) for item in sur_test_vector])
response = client.invoke_endpoint(EndpointName=endpoint_name,
                               ContentType='text/csv',
                               Body=body)
print(response["Body"].read())

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


b'{"predictions": [{"score": 0.7388485670089722}]}'


In [3]:
import pandas as pd
payload={"Pclass":"3","Name":"boung, Mr. Owen Harris","Sex":"male","Age":"22","SibSp":1,"Parch":0,"Ticket":"A/5 21171","Fare":7.25,
 "Cabin":"C85","Embarked":"S"}
print(type(payload))
df=pd.DataFrame(index=[0])
for item in payload:
    df[item]=payload[item]
print(df)

<class 'dict'>
  Pclass                    Name   Sex Age  SibSp  Parch     Ticket  Fare  \
0      3  boung, Mr. Owen Harris  male  22      1      0  A/5 21171  7.25   

  Cabin Embarked  
0   C85        S  


In [10]:
import numpy as np
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
df['cabin_multiple'] = df.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))
df['cabin_adv'] = df.Cabin.apply(lambda x: str(x)[0])
df['numeric_ticket'] = df.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
df['ticket_letters'] = df.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1]).replace('.','').replace('/','').lower() if len(x.split(' ')[:-1]) >0 else 0)
df['name_title'] = df.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())
df.Age = int(df.Age.fillna(df.Age.median()))
df.Fare = int(df.Fare.fillna(df.Fare.median()))
df.dropna(subset=['Embarked'],inplace = True)
df['norm_sibsp'] = int(np.log(df.SibSp+1))
df['norm_fare'] = int(np.log(df.Fare+1))
df.Pclass = df.Pclass.astype(str)
all_dummies = pd.get_dummies(df[['Pclass','Sex','Age','SibSp','Parch','norm_fare','Embarked','cabin_adv','cabin_multiple','numeric_ticket','name_title']])
#scaling
#print(encoded_df)
encoded_df = all_dummies.copy()
encoded_df
#encoded_df[['Age','SibSp','Parch','norm_fare']]= scale.fit_transform(encoded_df[['Age','SibSp','Parch','norm_fare']])
encoded_df

,Age,SibSp,Parch,norm_fare,cabin_multiple,numeric_ticket,Pclass_3,Sex_male,Embarked_S,cabin_adv_C,name_title_Mr
0,22,1,0,2,1,0,1,1,1,1,1


In [34]:
import pandas as pd
import json
import glob
df=pd.read_csv("results.csv")
df=df.iloc[:,1:]
json_list = json.loads(df.to_json(orient = "records"))
print(json_list[1])

{'Age': -0.101339932, 'SibSp': 0.431350239, 'Parch': -0.474325852, 'norm_fare': 2.009558261, 'cabin_multiple': 0, 'numeric_ticket': 0, 'Pclass_1': 1, 'Pclass_2': 0, 'Pclass_3': 0, 'Sex_female': 1, 'Sex_male': 0, 'Embarked_C': 0, 'Embarked_Q': 0, 'Embarked_S': 1, 'cabin_adv_A': 0, 'cabin_adv_B': 0, 'cabin_adv_C': 0, 'cabin_adv_D': 0, 'cabin_adv_E': 0, 'cabin_adv_F': 0, 'cabin_adv_G': 0, 'cabin_adv_T': 0, 'cabin_adv_n': 1, 'name_title_Capt': 0, 'name_title_Col': 0, 'name_title_Don': 0, 'name_title_Dr': 0, 'name_title_Jonkheer': 0, 'name_title_Lady': 0, 'name_title_Major': 0, 'name_title_Master': 0, 'name_title_Miss': 0, 'name_title_Mlle': 0, 'name_title_Mme': 0, 'name_title_Mr': 0, 'name_title_Mrs': 1, 'name_title_Ms': 0, 'name_title_Rev': 0, 'name_title_Sir': 0, 'name_title_the Countess': 0}


In [35]:
inferdt=json_list[0]
payld=[]
for indx in inferdt:
    payld.append(inferdt[indx])

In [36]:
import boto3
import json
endpoint_name='wi-mlops-titanic-ml-train-piln-lr-endpoint'
client = boto3.client('sagemaker-runtime')
sur_test_vector = [-1.4112866455793727,0.43135023895238417,2.006119339191895,1.89904574757844,2,1,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0]
nonsur_test_vector=[0.6692169576775447,-0.47519908120995513,-0.474325852103786,-0.3308153928167703,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0]
body = ','.join([str(item) for item in payld])
response = client.invoke_endpoint(EndpointName=endpoint_name,
                               ContentType='text/csv',
                               Body=body)
print(response["Body"].read())

b'{"predictions": [{"score": 0.4429289698600769}]}'


**Note:** Similarly you can change the test vector and can see how endpoint is working

**References** <br/>
- How to configure preprocessing jobs on step function (https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_CreateProcessingJob.html)
- How to create batch transform jobs
(https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_CreateTransformJob.html)
- Create training jobs
(https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_CreateTrainingJob.html)
- In case if team wants to bring any sagemaker component
(https://docs.aws.amazon.com/step-functions/latest/dg/connect-sagemaker.html)
- How to create lambda using state function
(https://docs.aws.amazon.com/step-functions/latest/dg/tutorial-creating-lambda-state-machine.html)

In [ ]:
#Clean up-- delete end point-


In [ ]:
#client = boto3.client("sagemaker", region_name=region)
#response=client.delete_endpoint(EndpointName=endpoint_name)

In [1]:
!aws s3 cp s3://wi-cred-datalake-dev-raw/titanic/feature/lr/baseline_prediction/baselinemodeldrift.csv.out .

download: s3://wi-cred-datalake-dev-raw/titanic/feature/lr/baseline_prediction/baselinemodeldrift.csv.out to ./baselinemodeldrift.csv.out


In [13]:
import pandas as pd
df=pd.read_csv("baselinemodeldrift.csv.out",header=None)

In [29]:
df1=df.iloc[:,[0,-1]]
df1.columns=["id","price"]
df1

,id,price
0,507,0.704926
1,852,-0.128371
2,436,0.946002
3,748,0.778718
4,682,0.641706
...,...,...
173,638,0.203213
174,81,0.105093
175,782,0.958502
176,7,0.432928


In [30]:
df1

,id,price
0,507,0.704926
1,852,-0.128371
2,436,0.946002
3,748,0.778718
4,682,0.641706
...,...,...
173,638,0.203213
174,81,0.105093
175,782,0.958502
176,7,0.432928
